In [4]:
# Import the required modules
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras.utils as image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
from tqdm import tqdm
import cv2
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
from keras.applications import VGG16
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Function to load the image, and convert into np arrays, to further train the model
def arrange_data(df):
    
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0]) #First column is the image paths
    
    for i in tqdm(range(len(img_paths))):       
        img = image.load_img(img_paths[i],target_size=(200,150,3))
        img = image.img_to_array(img)
        img = img/255
        image_data.append(img)
        
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:29])
    
    # Display the size
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y

In [14]:
pd.set_option('display.max_colwidth', None)

In [20]:
train_df.iloc[:,1:29]

,0,0.1,0.2,0.3,0.4,1,0.5,0.6,0.7,0.8,...,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25
0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4827,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4828,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4829,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4830,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#Read and print and call the fucntion
print("Processing train..")
train_df = pd.read_csv("Data/Train.csv", delimiter=" ")
X_train, Y_train = arrange_data (train_df)

print("Processing valid..")
val_df = pd.read_csv("Data/Valid.csv", delimiter=" ")
X_val, Y_val = arrange_data (val_df)

Processing train..


100%|█████████████████████████████████████████████████████████████████████████████| 4832/4832 [00:25<00:00, 191.68it/s]


Shape of images: (4832, 200, 150, 3)
Shape of labels: (4832, 28)
Processing valid..


100%|█████████████████████████████████████████████████████████████████████████████| 1863/1863 [00:10<00:00, 179.41it/s]


Shape of images: (1863, 200, 150, 3)
Shape of labels: (1863, 28)


In [9]:
#Number of classes
num_classes = 28  

#Model Declaration
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(200,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 196, 146, 16)      1216      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 98, 73, 16)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 98, 73, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 94, 69, 32)        12832     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 47, 34, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 47, 34, 32)       

In [10]:
#Model Training and Saving
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), batch_size=64)
model.save('Models/Model_6c.h5')

Epoch 1/10
 1/76 [..............................] - ETA: 5:50 - loss: 0.7436 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [9]:
# Number of Genres
num_classes = 28

#Model Declaration
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(200,150, 3))
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
    
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 12288)             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              12583936  
                                                                 
 dropout_6 (Dropout)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 28)                28700     
                                                                 
Total params: 27,327,324
Trainable params: 19,692,060
Non-trainable params: 7,635,264
_________________________________________________________________


In [10]:
#Model Training and Saving
model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, 
                         shear_range=0.15,horizontal_flip=True, fill_mode="nearest")

EPOCHS=10
BS = 64
history = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS),validation_data=(X_val, Y_val), 
                    steps_per_epoch=len(X_train) // BS, epochs=EPOCHS)
model.save('Models/Model_4d.h5')

Epoch 1/10
75/75 [==============================] - 1495s 20s/step - loss: 0.2703 - accuracy: 0.1932 - val_loss: 0.2398 - val_accuracy: 0.2088
Epoch 2/10
75/75 [==============================] - 1440s 19s/step - loss: 0.2430 - accuracy: 0.2043 - val_loss: 0.2364 - val_accuracy: 0.2088
Epoch 3/10
75/75 [==============================] - 1555s 21s/step - loss: 0.2393 - accuracy: 0.2108 - val_loss: 0.2335 - val_accuracy: 0.2088
Epoch 4/10
75/75 [==============================] - 1486s 20s/step - loss: 0.2353 - accuracy: 0.2175 - val_loss: 0.2332 - val_accuracy: 0.2088
Epoch 5/10
75/75 [==============================] - 1424s 19s/step - loss: 0.2328 - accuracy: 0.2217 - val_loss: 0.2387 - val_accuracy: 0.2442
Epoch 6/10
75/75 [==============================] - 1465s 20s/step - loss: 0.2304 - accuracy: 0.2181 - val_loss: 0.2335 - val_accuracy: 0.2083
Epoch 7/10
75/75 [==============================] - 1473s 20s/step - loss: 0.2287 - accuracy: 0.2215 - val_loss: 0.2373 - val_accuracy: 0.2142